In [1]:
import pandas as pd
import pickle
import json

In [2]:
# pip freeze > requirements.txt

In [3]:
filename = 'clf_model.sav'
best_clf = pickle.load(open(filename, 'rb'))
# score = best_clf.score(X_test, y_test)
best_clf

DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [4]:
with open('department_codes.json', 'r') as f:
    department_codes = json.load(f)

In [5]:
data = '''
{
"predict_delivery":
    [
        {
        "OrderNumber": "DL00-037866",
        "OrderDate": "2022-08-01", 
        "OrderAmount": 8524, 
        "DeliveryAttempt": 4, 
        "DeliveryDate": "2022-08-03",
        "Department": "Владикавказ"
        },
        {
        "OrderNumber": "DL00-037867",
        "OrderDate": "2022-07-01", 
        "OrderAmount": 2000, 
        "DeliveryAttempt": 0, 
        "DeliveryDate": "2022-07-03",
        "Department": "Москва"
        }       
    ]
}
'''

In [6]:
dict1 = json.loads(data)

In [7]:
dict1['predict_delivery']

[{'OrderNumber': 'DL00-037866',
  'OrderDate': '2022-08-01',
  'OrderAmount': 8524,
  'DeliveryAttempt': 4,
  'DeliveryDate': '2022-08-03',
  'Department': 'Владикавказ'},
 {'OrderNumber': 'DL00-037867',
  'OrderDate': '2022-07-01',
  'OrderAmount': 2000,
  'DeliveryAttempt': 0,
  'DeliveryDate': '2022-07-03',
  'Department': 'Москва'}]

In [8]:
df = pd.json_normalize(dict1['predict_delivery'])
df

,OrderNumber,OrderDate,OrderAmount,DeliveryAttempt,DeliveryDate,Department
0,DL00-037866,2022-08-01,8524,4,2022-08-03,Владикавказ
1,DL00-037867,2022-07-01,2000,0,2022-07-03,Москва


In [9]:
df[["OrderDate", "DeliveryDate"]] = df[["OrderDate", "DeliveryDate"]].apply(pd.to_datetime)

In [10]:
df['DaysDelivery'] = (df['DeliveryDate'] - df['OrderDate']).dt.days

In [11]:
df['Weekday'] = df['DeliveryDate'].dt.weekday

In [12]:
df['Department'] = df['Department'].apply(lambda x: department_codes[x])

In [13]:
df

,OrderNumber,OrderDate,OrderAmount,DeliveryAttempt,DeliveryDate,Department,DaysDelivery,Weekday
0,DL00-037866,2022-08-01,8524,4,2022-08-03,7,2,2
1,DL00-037867,2022-07-01,2000,0,2022-07-03,25,2,6


In [14]:
X = df[['DeliveryAttempt', 'DaysDelivery', 'Department', 'OrderAmount', 'Weekday']]
X

,DeliveryAttempt,DaysDelivery,Department,OrderAmount,Weekday
0,4,2,7,8524,2
1,0,2,25,2000,6


In [15]:
predict_proba_result = best_clf.predict_proba(X)

In [16]:
predict_proba_result_list = predict_proba_result.tolist()

In [17]:
result_list = list()
for i in range(len(predict_proba_result_list)):
    result_list.append({'OrderNumber': dict1['predict_delivery'][i]['OrderNumber'],
                       'delivery_prediction': round(predict_proba_result_list[i][1],2)})
result_list

[{'OrderNumber': 'DL00-037866', 'delivery_prediction': 0.92},
 {'OrderNumber': 'DL00-037867', 'delivery_prediction': 0.47}]